In [36]:
import tensorflow as tf
from tensorflow.keras import layers, models

#generator model

def build_generator():
  model = models.Sequential([
      layers.Dense(7*7*128, input_shape=(100,)),
      layers.LeakyReLU(),
      layers.Reshape((7, 7, 128)),
      layers.Conv2DTranspose(128,(4,4), strides=(2,2), padding='same'),
      layers.LeakyReLU(),
      layers.Conv2DTranspose(128,(4,4), strides=(2,2), padding='same'),
      layers.LeakyReLU(),
      layers.Conv2D(1,(7,7), padding='same', activation='sigmoid')
  ])

  return model

In [45]:
#discriminator model

def build_discriminator():
  model = models.Sequential([
      layers.Conv2D(64, (5,5), strides=(2,2), padding='same', input_shape=[28, 28, 1]),
      layers.LeakyReLU(),
      layers.Dropout(0.3),
      layers.Conv2D(128, (5,5),strides=(2,2),padding='same'),
      layers.LeakyReLU(),
      layers.Dropout(0.3),
      layers.Flatten(),
      layers.Dense(1)
  ])
  return model


In [46]:
#GAN model

def build_gan(generator, discriminator):
  discriminator.traniable = False
  model = models.Sequential([generator, discriminator])

  return model

In [47]:
#load MNIST data

mnist = tf.keras.datasets.mnist
(train_images,_),(_,_) = mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images-127.5) / 127.5 #normalize the image to [-1, -1]
#batch size
batch_size = 256
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(train_images.shape[0]).batch(batch_size)





In [48]:

#Intialize the model
generator = build_generator()
discriminator = build_discriminator()

gan = build_gan(generator, discriminator)


In [57]:

#Intialize the model
generator = build_generator()
discriminator = build_discriminator()

gan = build_gan(generator, discriminator)

#Define loss function
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits =True)

#Define optimizers
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

#Define training setUp
@tf.function
def train_step(images) :
  noise = tf.random.normal([batch_size, 100])
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    generated_images = generator(noise, training=True)
    real_output = discriminator(images, training=True)
    fake_output = discriminator(generated_images, training=True)
    gen_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
    disc_loss_real = cross_entropy(tf.ones_like(real_output),real_output)
    disc_loss_fake = cross_entropy(tf.zeros_like(fake_output),fake_output)
    disc_loss = disc_loss_real + disc_loss_fake

  gradient_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
  gradient_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
  generator_optimizer.apply_gradients(zip(gradient_of_generator, generator.trainable_variables))
  discriminator_optimizer.apply_gradients(zip(gradient_of_discriminator, discriminator.trainable_variables))


In [ ]:
  #traing loop
  epochs = 20
  for epoch in range(epochs):
    for image_batch in train_dataset:
      train_step(image_batch)
    print(f'Epoch{epoch+1}/{epochs}')